In [ ]:
# 03. Exploratory Data Analysis (EDA)

## Purpose
This notebook conducts comprehensive exploratory data analysis on the cleaned and feature-engineered yield curve dataset to uncover stylized patterns, evaluate time-series behavior, and determine appropriate forecasting horizons for machine learning models.

## Objectives
1. **Historical Trend Visualization** - Comprehensive visual analysis of yield curve evolution
2. **Stationarity Analysis** - Statistical tests for time series properties
3. **Forecast Horizon Strategy** - Empirical analysis to guide modeling approach
4. **Modeling Recommendations** - Evidence-based strategy for univariate vs multivariate forecasting

## Expected Outputs
- Complete visual documentation of yield curve dynamics
- Statistical assessment of data properties and stationarity
- Recommended forecasting horizon and modeling strategy
- High-quality figures saved for research publication

## Key Research Questions
- How do yield curve dynamics behave across different market regimes?
- What are the statistical properties of yield levels and derived features?
- Which forecasting horizons offer the best predictability?
- Should we model tenors independently or jointly?

---


In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
from pathlib import Path
import logging
import json
import pickle
from scipy import stats
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

# Statistical testing
from statsmodels.tsa.stattools import adfuller, kpss
from statsmodels.stats.diagnostic import acorr_ljungbox
from statsmodels.tsa.vector_ar.var_model import VAR
import statsmodels.api as sm

# Enhanced visualization
import matplotlib.dates as mdates
from matplotlib.animation import FuncAnimation
from mpl_toolkits.mplot3d import Axes3D
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

# Configuration
warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

# Setup logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

# Create directories for outputs
Path("../reports/figures").mkdir(parents=True, exist_ok=True)
Path("../reports/tables").mkdir(parents=True, exist_ok=True)

print("✅ Libraries imported successfully")
print(f"Analysis Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

# Set random seed for reproducibility
np.random.seed(42)


In [ ]:
## 1. Data Loading and Preparation

Load the feature-engineered dataset from Phase 2 or generate realistic sample data for comprehensive EDA analysis.


In [ ]:
def load_or_generate_data():
    """
    Load processed data from Phase 2 or generate realistic sample data for EDA.
    """
    # Try to load processed data from Phase 2
    processed_files = list(Path("../data/processed").glob("complete_dataset_*.csv"))
    
    if processed_files:
        # Load the most recent processed dataset
        latest_file = max(processed_files, key=lambda f: f.stat().st_mtime)
        print(f"📂 Loading processed data: {latest_file.name}")
        df = pd.read_csv(latest_file)
        df['date'] = pd.to_datetime(df['date'])
        print(f"✅ Loaded processed dataset with shape: {df.shape}")
        return df
    else:
        print("📂 No processed data found. Generating realistic sample data for EDA...")
        return generate_comprehensive_sample_data()

def generate_comprehensive_sample_data():
    """
    Generate comprehensive realistic yield curve and macro data for EDA demonstration.
    """
    # Create business day range (2000-2024)
    start_date = '2000-01-01'
    end_date = '2024-12-01'
    date_range = pd.bdate_range(start=start_date, end=end_date, freq='B')
    n_days = len(date_range)
    
    # Yield curve tenors
    tenors = ['1M', '3M', '6M', '1Y', '2Y', '3Y', '5Y', '7Y', '10Y', '20Y', '30Y']
    tenor_years = [1/12, 3/12, 6/12, 1, 2, 3, 5, 7, 10, 20, 30]
    
    print(f"🔄 Generating {n_days:,} observations for comprehensive EDA...")
    
    # Generate realistic market regimes
    time_factor = np.linspace(0, 1, n_days)
    
    # Create distinct market regimes with different characteristics
    # Regime 1: 2000-2007 (Pre-crisis, rising rates)
    # Regime 2: 2008-2015 (Crisis and QE, low rates)  
    # Regime 3: 2016-2019 (Normalization)
    # Regime 4: 2020-2024 (COVID and recovery)
    
    regime_breaks = [
        int(0.32 * n_days),  # ~2007
        int(0.64 * n_days),  # ~2015
        int(0.80 * n_days),  # ~2019
    ]
    
    # Generate level factor with regime characteristics
    level_base = np.concatenate([
        np.linspace(3.5, 5.5, regime_breaks[0]),                    # Rising pre-crisis
        np.linspace(5.5, 0.5, regime_breaks[1] - regime_breaks[0]), # Crisis/QE
        np.linspace(0.5, 2.5, regime_breaks[2] - regime_breaks[1]), # Normalization
        np.linspace(2.5, 1.0, n_days - regime_breaks[2])           # COVID/Recovery
    ])
    
    # Add random walk component
    level_factor = level_base + np.cumsum(np.random.normal(0, 0.015, n_days))
    
    # Generate slope factor (term structure steepness)
    slope_base = np.concatenate([
        np.linspace(1.0, 0.5, regime_breaks[0]),                    # Flattening pre-crisis
        np.linspace(0.5, 2.5, regime_breaks[1] - regime_breaks[0]), # Steepening during QE
        np.linspace(2.5, 1.0, regime_breaks[2] - regime_breaks[1]), # Normalization
        np.linspace(1.0, 1.8, n_days - regime_breaks[2])           # Steepening
    ])
    
    slope_factor = slope_base + np.cumsum(np.random.normal(0, 0.008, n_days))
    
    # Generate curvature factor
    curvature_factor = 0.1 * np.sin(2 * np.pi * time_factor * 8) + \
                      np.cumsum(np.random.normal(0, 0.005, n_days))
    
    # Generate yields with realistic term structure
    yields_data = {}
    base_rates = np.array([0.25, 0.5, 0.8, 1.2, 1.8, 2.2, 2.8, 3.1, 3.5, 4.0, 4.3])
    
    for i, (tenor, tenor_year) in enumerate(zip(tenors, tenor_years)):
        # Term structure loadings
        level_loading = 1.0
        slope_loading = np.log(tenor_year + 0.25)  # Log term structure
        curvature_loading = tenor_year * (10 - tenor_year) / 25  # Hump-shaped
        
        yields = (base_rates[i] + 
                 level_loading * level_factor +
                 slope_loading * slope_factor +
                 curvature_loading * curvature_factor +
                 np.random.normal(0, 0.03, n_days))  # Idiosyncratic noise
        
        # Ensure non-negative yields
        yields = np.maximum(yields, 0.01)
        yields_data[tenor] = yields
    
    # Create continuously compounded yields
    for tenor in tenors:
        yields_data[f'{tenor}_cc'] = np.log(1 + yields_data[tenor] / 100)
    
    # Generate derived yield curve features
    yields_data['yield_slope_10y2y'] = yields_data['10Y'] - yields_data['2Y']
    yields_data['yield_curvature'] = (yields_data['2Y'] + yields_data['30Y']) - 2 * yields_data['10Y']
    yields_data['yield_level'] = np.mean([yields_data[t] for t in tenors], axis=0)
    yields_data['yield_range'] = np.max([yields_data[t] for t in tenors], axis=0) - \
                                np.min([yields_data[t] for t in tenors], axis=0)
    
    # Generate PCA scores (simulate typical level, slope, curvature factors)
    yields_data['pca_factor_1'] = level_factor + np.random.normal(0, 0.1, n_days)
    yields_data['pca_factor_2'] = slope_factor + np.random.normal(0, 0.08, n_days)
    yields_data['pca_factor_3'] = curvature_factor + np.random.normal(0, 0.06, n_days)
    
    # Generate macro indicators
    # Fed Funds Rate
    fed_funds = np.concatenate([
        np.linspace(6.0, 5.25, regime_breaks[0]),
        np.linspace(5.25, 0.25, regime_breaks[1] - regime_breaks[0]),
        np.linspace(0.25, 2.5, regime_breaks[2] - regime_breaks[1]),
        np.linspace(2.5, 0.25, n_days - regime_breaks[2])
    ]) + np.cumsum(np.random.normal(0, 0.01, n_days))
    yields_data['fed_funds_rate'] = np.maximum(fed_funds, 0.0)
    
    # VIX
    vix_base = 20 + 15 * np.sin(2 * np.pi * time_factor * 12)  # Cyclical volatility
    crisis_spikes = np.zeros(n_days)
    crisis_spikes[int(0.35*n_days):int(0.37*n_days)] = 40  # 2008 crisis
    crisis_spikes[int(0.85*n_days):int(0.86*n_days)] = 60  # 2020 COVID
    yields_data['vix'] = np.maximum(vix_base + crisis_spikes + np.random.normal(0, 3, n_days), 5)
    
    # Unemployment Rate
    unemployment = np.concatenate([
        4.0 + 1.0 * np.sin(np.linspace(0, 4*np.pi, regime_breaks[0])),
        np.linspace(5.0, 10.0, int(0.1*(regime_breaks[1] - regime_breaks[0]))) + \
        np.linspace(10.0, 5.0, int(0.9*(regime_breaks[1] - regime_breaks[0]))),
        np.linspace(5.0, 3.5, regime_breaks[2] - regime_breaks[1]),
        np.concatenate([np.linspace(3.5, 14.8, int(0.2*(n_days - regime_breaks[2]))),
                       np.linspace(14.8, 3.8, int(0.8*(n_days - regime_breaks[2])))])
    ])
    yields_data['unemployment_rate'] = np.clip(unemployment + np.random.normal(0, 0.1, n_days), 2, 16)
    
    # Create DataFrame
    df = pd.DataFrame(yields_data, index=date_range)
    df.index.name = 'date'
    df = df.reset_index()
    
    # Add market regime labels for analysis
    regime_labels = np.concatenate([
        np.repeat('Pre-Crisis (2000-2007)', regime_breaks[0]),
        np.repeat('Crisis/QE (2008-2015)', regime_breaks[1] - regime_breaks[0]),
        np.repeat('Normalization (2016-2019)', regime_breaks[2] - regime_breaks[1]),
        np.repeat('COVID/Recovery (2020-2024)', n_days - regime_breaks[2])
    ])
    df['market_regime'] = regime_labels
    
    print(f"✅ Generated comprehensive sample dataset with shape: {df.shape}")
    return df

# Load the data
print("🔄 Loading data for EDA analysis...")
df = load_or_generate_data()

# Display basic information
print(f"\n📊 Dataset Overview:")
print(f"Date range: {df['date'].min()} to {df['date'].max()}")
print(f"Total observations: {len(df):,}")
print(f"Number of variables: {len(df.columns)}")

# Display first few rows
print(f"\n📋 First 5 rows:")
yield_cols = ['1M', '3M', '6M', '1Y', '2Y', '5Y', '10Y', '30Y']
display_cols = ['date'] + [col for col in yield_cols if col in df.columns]
if 'market_regime' in df.columns:
    display_cols.append('market_regime')
print(df[display_cols].head())


In [ ]:
## 2. Historical Trend Visualization

This section provides comprehensive visual analysis of yield curve evolution across different market regimes, highlighting structural behaviors, steepening/flattening patterns, and key economic events.


In [ ]:
# Define key variables for analysis
yield_tenors = ['1M', '3M', '6M', '1Y', '2Y', '3Y', '5Y', '7Y', '10Y', '20Y', '30Y']
available_tenors = [tenor for tenor in yield_tenors if tenor in df.columns]

print(f"📊 Available yield tenors for analysis: {available_tenors}")

# Set up enhanced plotting parameters
plt.rcParams['figure.figsize'] = (15, 10)
plt.rcParams['font.size'] = 12
plt.rcParams['axes.titlesize'] = 16
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12
plt.rcParams['legend.fontsize'] = 12

def save_figure(fig, filename, dpi=300):
    """Save figure with high quality for publication."""
    filepath = f"../reports/figures/{filename}"
    fig.savefig(filepath, dpi=dpi, bbox_inches='tight', facecolor='white')
    print(f"💾 Saved: {filepath}")

print("✅ Plotting configuration set")


In [ ]:
### 2.1 Complete Yield Curve Time Series Evolution

print("🔄 Creating comprehensive yield curve time series visualization...")

# Create multi-panel plot showing all tenors
fig, axes = plt.subplots(3, 1, figsize=(20, 18))

# Panel 1: Short-term rates (1M - 2Y)
short_tenors = [t for t in ['1M', '3M', '6M', '1Y', '2Y'] if t in available_tenors]
for tenor in short_tenors:
    axes[0].plot(df['date'], df[tenor], label=f'{tenor}', linewidth=1.5, alpha=0.8)

axes[0].set_title('Short-Term Treasury Yields (1M - 2Y)', fontsize=16, fontweight='bold')
axes[0].set_ylabel('Yield (%)', fontsize=14)
axes[0].legend(loc='upper right')
axes[0].grid(True, alpha=0.3)
axes[0].set_xlim(df['date'].min(), df['date'].max())

# Panel 2: Medium-term rates (3Y - 10Y)  
medium_tenors = [t for t in ['3Y', '5Y', '7Y', '10Y'] if t in available_tenors]
for tenor in medium_tenors:
    axes[1].plot(df['date'], df[tenor], label=f'{tenor}', linewidth=1.5, alpha=0.8)

axes[1].set_title('Medium-Term Treasury Yields (3Y - 10Y)', fontsize=16, fontweight='bold')
axes[1].set_ylabel('Yield (%)', fontsize=14)
axes[1].legend(loc='upper right')
axes[1].grid(True, alpha=0.3)
axes[1].set_xlim(df['date'].min(), df['date'].max())

# Panel 3: Long-term rates (20Y - 30Y)
long_tenors = [t for t in ['20Y', '30Y'] if t in available_tenors]
for tenor in long_tenors:
    axes[2].plot(df['date'], df[tenor], label=f'{tenor}', linewidth=1.5, alpha=0.8)

axes[2].set_title('Long-Term Treasury Yields (20Y - 30Y)', fontsize=16, fontweight='bold')
axes[2].set_ylabel('Yield (%)', fontsize=14)
axes[2].set_xlabel('Date', fontsize=14)
axes[2].legend(loc='upper right')
axes[2].grid(True, alpha=0.3)
axes[2].set_xlim(df['date'].min(), df['date'].max())

# Format x-axis for all panels
for ax in axes:
    ax.xaxis.set_major_locator(mdates.YearLocator(5))
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y'))
    ax.tick_params(axis='x', rotation=45)

# Add market regime shading if available
if 'market_regime' in df.columns:
    regime_changes = df.groupby('market_regime')['date'].agg(['min', 'max'])
    colors = ['lightblue', 'lightcoral', 'lightgreen', 'lightyellow']
    
    for i, (regime, dates) in enumerate(regime_changes.iterrows()):
        for ax in axes:
            ax.axvspan(dates['min'], dates['max'], alpha=0.2, color=colors[i % len(colors)], 
                      label=regime if ax == axes[0] else "")
    
    # Add regime legend to top panel
    axes[0].legend(loc='upper right', bbox_to_anchor=(1.15, 1))

plt.tight_layout()
save_figure(fig, 'yield_curve_time_series_evolution.png')
plt.show()

print("✅ Yield curve time series visualization completed")


In [ ]:
### 2.2 Yield Curve Shape Evolution Snapshots

print("🔄 Creating yield curve shape evolution snapshots...")

# Select representative dates for different market conditions
if 'market_regime' in df.columns:
    snapshot_dates = []
    for regime in df['market_regime'].unique():
        regime_data = df[df['market_regime'] == regime]
        mid_date = regime_data['date'].iloc[len(regime_data)//2]
        snapshot_dates.append((mid_date, regime))
else:
    # Select evenly spaced dates
    n_snapshots = 4
    date_indices = np.linspace(0, len(df)-1, n_snapshots, dtype=int)
    snapshot_dates = [(df.iloc[i]['date'], f'Period {i+1}') for i in date_indices]

fig, axes = plt.subplots(2, 2, figsize=(18, 12))
axes = axes.flatten()

tenor_years = [1/12, 3/12, 6/12, 1, 2, 3, 5, 7, 10, 20, 30][:len(available_tenors)]

for i, (date, label) in enumerate(snapshot_dates):
    if i >= 4:  # Limit to 4 snapshots
        break
        
    # Find closest date in data
    closest_idx = (df['date'] - date).abs().idxmin()
    snapshot_data = df.iloc[closest_idx]
    
    yields = [snapshot_data[tenor] for tenor in available_tenors]
    
    axes[i].plot(tenor_years, yields, 'o-', linewidth=2.5, markersize=8, alpha=0.8)
    axes[i].set_title(f'{label}\\n{snapshot_data["date"].strftime("%Y-%m-%d")}', 
                     fontsize=14, fontweight='bold')
    axes[i].set_xlabel('Maturity (Years)', fontsize=12)
    axes[i].set_ylabel('Yield (%)', fontsize=12)
    axes[i].grid(True, alpha=0.3)
    axes[i].set_xscale('log')
    
    # Customize x-axis ticks
    axes[i].set_xticks(tenor_years)
    axes[i].set_xticklabels(available_tenors, rotation=45)
    
    # Add yield level annotation
    avg_yield = np.mean(yields)
    axes[i].text(0.7, 0.9, f'Avg Yield: {avg_yield:.2f}%', 
                transform=axes[i].transAxes, fontsize=11,
                bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8))

plt.suptitle('Yield Curve Shape Evolution Across Market Regimes', 
             fontsize=18, fontweight='bold', y=0.98)
plt.tight_layout()
save_figure(fig, 'yield_curve_shape_snapshots.png')
plt.show()

print("✅ Yield curve shape snapshots completed")


In [ ]:
### 2.3 Slope and Curvature Analysis

print("🔄 Creating slope and curvature analysis...")

# Create slope and curvature plots
fig, axes = plt.subplots(3, 1, figsize=(20, 15))

# Calculate derived measures if not already present
if 'yield_slope_10y2y' not in df.columns and '10Y' in df.columns and '2Y' in df.columns:
    df['yield_slope_10y2y'] = df['10Y'] - df['2Y']

if 'yield_curvature' not in df.columns and all(t in df.columns for t in ['2Y', '10Y', '30Y']):
    df['yield_curvature'] = (df['2Y'] + df['30Y']) - 2 * df['10Y']

# Panel 1: 10Y-2Y Slope
if 'yield_slope_10y2y' in df.columns:
    axes[0].plot(df['date'], df['yield_slope_10y2y'], color='darkblue', linewidth=2, alpha=0.8)
    axes[0].axhline(y=0, color='red', linestyle='--', alpha=0.7, linewidth=1)
    axes[0].fill_between(df['date'], df['yield_slope_10y2y'], 0, 
                        where=(df['yield_slope_10y2y'] < 0), color='red', alpha=0.3, 
                        label='Inversion')
    axes[0].fill_between(df['date'], df['yield_slope_10y2y'], 0,
                        where=(df['yield_slope_10y2y'] >= 0), color='blue', alpha=0.3,
                        label='Normal')
    
    axes[0].set_title('Yield Curve Slope (10Y - 2Y): Steepening, Flattening, and Inversions', 
                     fontsize=16, fontweight='bold')
    axes[0].set_ylabel('Slope (bp)', fontsize=14)
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)
    
    # Calculate inversion statistics
    inversions = df['yield_slope_10y2y'] < 0
    inversion_pct = inversions.mean() * 100
    axes[0].text(0.02, 0.95, f'Inversion periods: {inversion_pct:.1f}% of time',
                transform=axes[0].transAxes, fontsize=12,
                bbox=dict(boxstyle='round', facecolor='yellow', alpha=0.8))

# Panel 2: Curvature
if 'yield_curvature' in df.columns:
    axes[1].plot(df['date'], df['yield_curvature'], color='darkgreen', linewidth=2, alpha=0.8)
    axes[1].axhline(y=0, color='red', linestyle='--', alpha=0.7, linewidth=1)
    
    axes[1].set_title('Yield Curve Curvature: (2Y + 30Y) - 2×(10Y)', 
                     fontsize=16, fontweight='bold')
    axes[1].set_ylabel('Curvature (bp)', fontsize=14)
    axes[1].grid(True, alpha=0.3)

# Panel 3: Combined slope and level
if all(col in df.columns for col in ['10Y', 'yield_slope_10y2y']):
    # Create scatter plot of level vs slope
    scatter = axes[2].scatter(df['10Y'], df['yield_slope_10y2y'], 
                            c=range(len(df)), cmap='viridis', alpha=0.6, s=20)
    axes[2].set_xlabel('10-Year Yield Level (%)', fontsize=14)
    axes[2].set_ylabel('Yield Curve Slope (10Y-2Y, bp)', fontsize=14)
    axes[2].set_title('Yield Level vs Slope Relationship (Color = Time)', 
                     fontsize=16, fontweight='bold')
    axes[2].grid(True, alpha=0.3)
    
    # Add colorbar
    cbar = plt.colorbar(scatter, ax=axes[2])
    cbar.set_label('Time (darker = more recent)', fontsize=12)

# Format dates
for ax in axes[:2]:  # Skip scatter plot
    ax.xaxis.set_major_locator(mdates.YearLocator(5))
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y'))
    ax.tick_params(axis='x', rotation=45)

plt.tight_layout()
save_figure(fig, 'slope_curvature_analysis.png')
plt.show()

# Print slope and curvature statistics
if 'yield_slope_10y2y' in df.columns:
    slope_stats = df['yield_slope_10y2y'].describe()
    print("📊 Yield Slope (10Y-2Y) Statistics:")
    print(slope_stats.round(2))
    
    print(f"\n🔍 Key Slope Insights:")
    print(f"• Average slope: {slope_stats['mean']:.1f} bp")
    print(f"• Slope volatility: {slope_stats['std']:.1f} bp")
    print(f"• Minimum slope (deepest inversion): {slope_stats['min']:.1f} bp")
    print(f"• Maximum slope (steepest): {slope_stats['max']:.1f} bp")

print("✅ Slope and curvature analysis completed")


In [ ]:
### 2.4 PCA Component Analysis

print("🔄 Creating PCA component visualization...")

# Check if PCA factors are available or compute them
pca_factors = [col for col in df.columns if col.startswith('pca_factor_')]

if not pca_factors and len(available_tenors) >= 3:
    print("📊 Computing PCA on yield curve...")
    
    # Prepare yield data for PCA
    yield_data = df[available_tenors].dropna()
    
    # Standardize before PCA
    scaler = StandardScaler()
    yield_scaled = scaler.fit_transform(yield_data)
    
    # Apply PCA
    pca = PCA(n_components=min(5, len(available_tenors)))
    pca_scores = pca.fit_transform(yield_scaled)
    
    # Add PCA scores to dataframe
    valid_indices = yield_data.index
    for i in range(pca_scores.shape[1]):
        df.loc[valid_indices, f'pca_factor_{i+1}'] = pca_scores[:, i]
    
    pca_factors = [f'pca_factor_{i+1}' for i in range(pca_scores.shape[1])]
    
    print(f"✅ Computed PCA with {len(pca_factors)} components")
    print(f"Explained variance ratio: {pca.explained_variance_ratio_[:3].round(3)}")

# Create PCA visualization
if pca_factors:
    n_factors = min(3, len(pca_factors))  # Show first 3 factors
    fig, axes = plt.subplots(n_factors, 1, figsize=(20, 5*n_factors))
    
    if n_factors == 1:
        axes = [axes]
    
    factor_names = ['Level Factor', 'Slope Factor', 'Curvature Factor']
    colors = ['darkblue', 'darkred', 'darkgreen']
    
    for i in range(n_factors):
        factor_col = pca_factors[i]
        if factor_col in df.columns:
            # Remove NaN values for plotting
            factor_data = df[['date', factor_col]].dropna()
            
            axes[i].plot(factor_data['date'], factor_data[factor_col], 
                        color=colors[i], linewidth=2, alpha=0.8)
            axes[i].set_title(f'PCA {factor_names[i]} - Component {i+1}', 
                             fontsize=16, fontweight='bold')
            axes[i].set_ylabel('Factor Score', fontsize=14)
            axes[i].grid(True, alpha=0.3)
            
            # Add zero line
            axes[i].axhline(y=0, color='black', linestyle='--', alpha=0.5)
            
            # Format dates
            axes[i].xaxis.set_major_locator(mdates.YearLocator(5))
            axes[i].xaxis.set_major_formatter(mdates.DateFormatter('%Y'))
            axes[i].tick_params(axis='x', rotation=45)
            
            # Add statistical summary
            factor_stats = factor_data[factor_col]
            axes[i].text(0.02, 0.95, 
                        f'Mean: {factor_stats.mean():.3f}, Std: {factor_stats.std():.3f}',
                        transform=axes[i].transAxes, fontsize=11,
                        bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8))
    
    # Set x-label for bottom plot
    if n_factors > 0:
        axes[-1].set_xlabel('Date', fontsize=14)
    
    plt.tight_layout()
    save_figure(fig, 'pca_factor_evolution.png')
    plt.show()
    
    # Create factor correlation heatmap
    if len(pca_factors) >= 2:
        pca_data = df[pca_factors].dropna()
        
        fig, ax = plt.subplots(figsize=(10, 8))
        
        # Calculate correlation matrix
        corr_matrix = pca_data.corr()
        
        # Create heatmap
        sns.heatmap(corr_matrix, annot=True, cmap='RdBu_r', center=0,
                   square=True, fmt='.3f', cbar_kws={'shrink': 0.8}, ax=ax)
        
        ax.set_title('PCA Factor Correlation Matrix', fontsize=16, fontweight='bold')
        plt.tight_layout()
        save_figure(fig, 'pca_factor_correlations.png')
        plt.show()

print("✅ PCA component analysis completed")


In [ ]:
## 3. Stationarity and Statistical Properties

This section conducts comprehensive statistical tests to understand the time series properties of yield levels and derived features. This analysis is crucial for model selection and feature engineering decisions.


In [ ]:
### 3.1 Stationarity Tests: ADF and KPSS

print("🔄 Conducting comprehensive stationarity analysis...")

def perform_stationarity_tests(series, series_name):
    """
    Perform ADF and KPSS tests on a time series.
    
    Returns dictionary with test results and interpretation.
    """
    # Remove NaN values
    clean_series = series.dropna()
    
    if len(clean_series) < 50:  # Not enough data
        return None
    
    results = {
        'series_name': series_name,
        'observations': len(clean_series)
    }
    
    # Augmented Dickey-Fuller test
    # H0: Series has unit root (non-stationary)
    # H1: Series is stationary
    try:
        adf_result = adfuller(clean_series, autolag='AIC')
        results['adf'] = {
            'statistic': adf_result[0],
            'p_value': adf_result[1],
            'critical_values': adf_result[4],
            'used_lag': adf_result[2],
            'is_stationary': adf_result[1] < 0.05
        }
    except Exception as e:
        results['adf'] = {'error': str(e)}
    
    # KPSS test
    # H0: Series is stationary
    # H1: Series has unit root (non-stationary)
    try:
        kpss_result = kpss(clean_series, regression='c', nlags='auto')
        results['kpss'] = {
            'statistic': kpss_result[0],
            'p_value': kpss_result[1],
            'critical_values': kpss_result[3],
            'used_lag': kpss_result[2],
            'is_stationary': kpss_result[1] > 0.05
        }
    except Exception as e:
        results['kpss'] = {'error': str(e)}
    
    # Combined interpretation
    if 'adf' in results and 'kpss' in results and 'error' not in results['adf'] and 'error' not in results['kpss']:
        adf_stat = results['adf']['is_stationary']
        kpss_stat = results['kpss']['is_stationary']
        
        if adf_stat and kpss_stat:
            results['interpretation'] = 'Stationary'
        elif not adf_stat and not kpss_stat:
            results['interpretation'] = 'Non-stationary'
        elif adf_stat and not kpss_stat:
            results['interpretation'] = 'Difference-stationary'
        else:
            results['interpretation'] = 'Trend-stationary'
    
    return results

# Test variables for stationarity
test_variables = []

# Add yield levels
test_variables.extend([(tenor, f'{tenor} Yield Level') for tenor in available_tenors])

# Add derived features if available
derived_features = [
    ('yield_slope_10y2y', '10Y-2Y Slope'),
    ('yield_curvature', 'Yield Curvature'), 
    ('yield_level', 'Average Yield Level'),
    ('fed_funds_rate', 'Fed Funds Rate'),
    ('vix', 'VIX'),
    ('unemployment_rate', 'Unemployment Rate')
]

for col, name in derived_features:
    if col in df.columns:
        test_variables.append((col, name))

# Add PCA factors
pca_factors = [col for col in df.columns if col.startswith('pca_factor_')]
for factor in pca_factors[:3]:  # Test first 3 PCA factors
    test_variables.append((factor, f'PCA {factor.split("_")[-1]}'))

print(f"📊 Testing {len(test_variables)} variables for stationarity...")

# Perform tests
stationarity_results = []
for col, name in test_variables:
    if col in df.columns:
        result = perform_stationarity_tests(df[col], name)
        if result:
            stationarity_results.append(result)

print(f"✅ Completed stationarity tests for {len(stationarity_results)} variables")


In [ ]:
### 3.2 Stationarity Results Summary

# Create comprehensive results table
stationarity_df = []

for result in stationarity_results:
    if 'adf' in result and 'kpss' in result and 'error' not in result['adf'] and 'error' not in result['kpss']:
        row = {
            'Variable': result['series_name'],
            'Observations': result['observations'],
            'ADF Statistic': result['adf']['statistic'],
            'ADF p-value': result['adf']['p_value'],
            'ADF Stationary': '✓' if result['adf']['is_stationary'] else '✗',
            'KPSS Statistic': result['kpss']['statistic'],
            'KPSS p-value': result['kpss']['p_value'], 
            'KPSS Stationary': '✓' if result['kpss']['is_stationary'] else '✗',
            'Interpretation': result.get('interpretation', 'Unclear')
        }
        stationarity_df.append(row)

stationarity_df = pd.DataFrame(stationarity_df)

# Display results
print("📊 STATIONARITY TEST RESULTS")
print("="*80)
print(stationarity_df.round(4))

# Create visualization of stationarity results
if len(stationarity_df) > 0:
    fig, axes = plt.subplots(2, 2, figsize=(20, 12))
    
    # Plot 1: ADF Test Statistics
    axes[0,0].barh(range(len(stationarity_df)), stationarity_df['ADF Statistic'], alpha=0.7)
    axes[0,0].axvline(x=stationarity_df['ADF Statistic'].quantile(0.05), color='red', 
                     linestyle='--', label='5% Critical Value (approx)')
    axes[0,0].set_yticks(range(len(stationarity_df)))
    axes[0,0].set_yticklabels(stationarity_df['Variable'], fontsize=10)
    axes[0,0].set_xlabel('ADF Test Statistic')
    axes[0,0].set_title('Augmented Dickey-Fuller Test Statistics\\n(More negative = More stationary)')
    axes[0,0].legend()
    axes[0,0].grid(True, alpha=0.3)
    
    # Plot 2: KPSS Test Statistics  
    axes[0,1].barh(range(len(stationarity_df)), stationarity_df['KPSS Statistic'], alpha=0.7, color='orange')
    axes[0,1].axvline(x=0.463, color='red', linestyle='--', label='5% Critical Value')  # KPSS 5% critical value
    axes[0,1].set_yticks(range(len(stationarity_df)))
    axes[0,1].set_yticklabels(stationarity_df['Variable'], fontsize=10)
    axes[0,1].set_xlabel('KPSS Test Statistic')
    axes[0,1].set_title('KPSS Test Statistics\\n(Lower = More stationary)')
    axes[0,1].legend()
    axes[0,1].grid(True, alpha=0.3)
    
    # Plot 3: P-values comparison
    x = range(len(stationarity_df))
    width = 0.35
    axes[1,0].bar([i - width/2 for i in x], stationarity_df['ADF p-value'], width, 
                 label='ADF p-value', alpha=0.7)
    axes[1,0].bar([i + width/2 for i in x], stationarity_df['KPSS p-value'], width,
                 label='KPSS p-value', alpha=0.7)
    axes[1,0].axhline(y=0.05, color='red', linestyle='--', label='5% Significance Level')
    axes[1,0].set_xlabel('Variables')
    axes[1,0].set_ylabel('p-value')
    axes[1,0].set_title('Stationarity Test p-values')
    axes[1,0].set_xticks(x)
    axes[1,0].set_xticklabels(stationarity_df['Variable'], rotation=45, ha='right')
    axes[1,0].legend()
    axes[1,0].grid(True, alpha=0.3)
    
    # Plot 4: Interpretation summary
    interpretation_counts = stationarity_df['Interpretation'].value_counts()
    colors = plt.cm.Set3(np.linspace(0, 1, len(interpretation_counts)))
    wedges, texts, autotexts = axes[1,1].pie(interpretation_counts.values, 
                                            labels=interpretation_counts.index,
                                            autopct='%1.1f%%', 
                                            colors=colors)
    axes[1,1].set_title('Stationarity Interpretation Summary')
    
    plt.tight_layout()
    save_figure(fig, 'stationarity_test_results.png')
    plt.show()

# Summary statistics
print(f"\n📈 STATIONARITY SUMMARY:")
if len(stationarity_df) > 0:
    stationary_vars = stationarity_df[stationarity_df['Interpretation'] == 'Stationary']
    non_stationary_vars = stationarity_df[stationarity_df['Interpretation'] == 'Non-stationary']
    
    print(f"• Stationary variables: {len(stationary_vars)} ({len(stationary_vars)/len(stationarity_df)*100:.1f}%)")
    print(f"• Non-stationary variables: {len(non_stationary_vars)} ({len(non_stationary_vars)/len(stationarity_df)*100:.1f}%)")
    
    if len(stationary_vars) > 0:
        print(f"\n✅ Stationary variables:")
        for var in stationary_vars['Variable']:
            print(f"   • {var}")
    
    if len(non_stationary_vars) > 0:
        print(f"\n⚠️  Non-stationary variables (may need differencing):")
        for var in non_stationary_vars['Variable']:
            print(f"   • {var}")

print("✅ Stationarity analysis completed")


In [ ]:
### 3.3 Descriptive Statistics and Distributions

print("🔄 Computing comprehensive descriptive statistics...")

# Select key variables for detailed analysis
analysis_vars = []

# Add yield levels
analysis_vars.extend(available_tenors)

# Add key derived features
key_features = ['yield_slope_10y2y', 'yield_curvature', 'yield_level', 'fed_funds_rate', 'vix']
analysis_vars.extend([var for var in key_features if var in df.columns])

# Add first 3 PCA factors
pca_vars = [col for col in df.columns if col.startswith('pca_factor_')][:3]
analysis_vars.extend(pca_vars)

# Compute descriptive statistics
desc_stats = df[analysis_vars].describe()

# Add skewness and kurtosis
from scipy.stats import skew, kurtosis
desc_stats.loc['skewness'] = df[analysis_vars].apply(lambda x: skew(x.dropna()))
desc_stats.loc['kurtosis'] = df[analysis_vars].apply(lambda x: kurtosis(x.dropna()))

print("📊 DESCRIPTIVE STATISTICS")
print("="*100)
print(desc_stats.round(4))

# Create distribution plots
n_vars = len(analysis_vars)
n_cols = 4
n_rows = (n_vars + n_cols - 1) // n_cols

fig, axes = plt.subplots(n_rows, n_cols, figsize=(20, 5*n_rows))
axes = axes.flatten() if n_rows > 1 else [axes] if n_cols == 1 else axes

for i, var in enumerate(analysis_vars):
    if i < len(axes) and var in df.columns:
        data = df[var].dropna()
        
        # Create histogram with normal overlay
        axes[i].hist(data, bins=50, density=True, alpha=0.7, color='skyblue', edgecolor='black')
        
        # Fit normal distribution for comparison
        mu, sigma = stats.norm.fit(data)
        x = np.linspace(data.min(), data.max(), 100)
        axes[i].plot(x, stats.norm.pdf(x, mu, sigma), 'r-', linewidth=2, label='Normal fit')
        
        axes[i].set_title(f'{var}\\nSkew: {skew(data):.2f}, Kurt: {kurtosis(data):.2f}', 
                         fontsize=12)
        axes[i].set_xlabel('Value')
        axes[i].set_ylabel('Density')
        axes[i].legend()
        axes[i].grid(True, alpha=0.3)

# Hide empty subplots
for i in range(len(analysis_vars), len(axes)):
    axes[i].set_visible(False)

plt.suptitle('Variable Distributions with Normal Distribution Overlay', 
             fontsize=16, fontweight='bold')
plt.tight_layout()
save_figure(fig, 'variable_distributions.png')
plt.show()

# Normality tests
print(f"\n📊 NORMALITY TESTS (Shapiro-Wilk)")
print("="*60)

normality_results = []
for var in analysis_vars[:8]:  # Limit to first 8 variables due to sample size constraints
    if var in df.columns:
        data = df[var].dropna()
        if len(data) > 10 and len(data) <= 5000:  # Shapiro-Wilk limitations
            stat, p_value = stats.shapiro(data[:5000])  # Use sample if too large
            is_normal = p_value > 0.05
            normality_results.append({
                'Variable': var,
                'Statistic': stat,
                'p-value': p_value,
                'Normal': '✓' if is_normal else '✗'
            })

if normality_results:
    normality_df = pd.DataFrame(normality_results)
    print(normality_df.round(6))

print("✅ Descriptive statistics analysis completed")


In [ ]:
## 4. Forecast Horizon Strategy Analysis

This section analyzes predictive signals, autocorrelations, and volatility across different time horizons to guide forecasting strategy and model selection decisions.


In [ ]:
### 4.1 Autocorrelation Analysis

print("🔄 Analyzing autocorrelation patterns across forecast horizons...")

def analyze_autocorrelation(series, lags=50, series_name="Series"):
    """Analyze autocorrelation structure of a time series."""
    clean_series = series.dropna()
    
    if len(clean_series) < lags + 10:
        return None
    
    # Calculate autocorrelations
    autocorrs = [clean_series.autocorr(lag=i) for i in range(1, lags+1)]
    
    # Calculate partial autocorrelations using statsmodels
    try:
        from statsmodels.tsa.stattools import pacf
        partial_autocorrs = pacf(clean_series, nlags=lags, method='yw')[1:]  # Skip lag 0
    except:
        partial_autocorrs = None
    
    return {
        'name': series_name,
        'autocorr': autocorrs,
        'partial_autocorr': partial_autocorrs,
        'significant_lags': [i+1 for i, ac in enumerate(autocorrs) if abs(ac) > 0.05]
    }

# Analyze key variables
target_vars = ['10Y', '2Y', '5Y', 'yield_slope_10y2y', 'pca_factor_1']
autocorr_results = {}

for var in target_vars:
    if var in df.columns:
        result = analyze_autocorrelation(df[var], lags=40, series_name=var)
        if result:
            autocorr_results[var] = result

# Plot autocorrelation functions
if autocorr_results:
    n_vars = len(autocorr_results)
    fig, axes = plt.subplots(n_vars, 2, figsize=(20, 4*n_vars))
    
    if n_vars == 1:
        axes = axes.reshape(1, -1)
    
    for i, (var, result) in enumerate(autocorr_results.items()):
        lags = range(1, len(result['autocorr']) + 1)
        
        # Autocorrelation function
        axes[i, 0].plot(lags, result['autocorr'], 'b-', alpha=0.8, linewidth=2)
        axes[i, 0].axhline(y=0, color='black', linestyle='-', alpha=0.5)
        axes[i, 0].axhline(y=0.05, color='red', linestyle='--', alpha=0.7, label='5% threshold')
        axes[i, 0].axhline(y=-0.05, color='red', linestyle='--', alpha=0.7)
        axes[i, 0].set_title(f'{var} - Autocorrelation Function', fontweight='bold')
        axes[i, 0].set_xlabel('Lag (days)')
        axes[i, 0].set_ylabel('Autocorrelation')
        axes[i, 0].grid(True, alpha=0.3)
        axes[i, 0].legend()
        
        # Partial autocorrelation function
        if result['partial_autocorr'] is not None:
            axes[i, 1].plot(lags, result['partial_autocorr'], 'r-', alpha=0.8, linewidth=2)
            axes[i, 1].axhline(y=0, color='black', linestyle='-', alpha=0.5)
            axes[i, 1].axhline(y=0.05, color='red', linestyle='--', alpha=0.7, label='5% threshold')
            axes[i, 1].axhline(y=-0.05, color='red', linestyle='--', alpha=0.7)
            axes[i, 1].set_title(f'{var} - Partial Autocorrelation Function', fontweight='bold')
            axes[i, 1].set_xlabel('Lag (days)')
            axes[i, 1].set_ylabel('Partial Autocorrelation')
            axes[i, 1].grid(True, alpha=0.3)
            axes[i, 1].legend()
    
    plt.tight_layout()
    save_figure(fig, 'autocorrelation_analysis.png')
    plt.show()
    
    # Print autocorrelation insights
    print("📊 AUTOCORRELATION INSIGHTS:")
    print("="*50)
    for var, result in autocorr_results.items():
        sig_lags = result['significant_lags'][:10]  # First 10 significant lags
        print(f"{var}:")
        print(f"  • Significant lags (>5%): {sig_lags}")
        print(f"  • 1-day autocorr: {result['autocorr'][0]:.3f}")
        print(f"  • 5-day autocorr: {result['autocorr'][4]:.3f}" if len(result['autocorr']) > 4 else "")
        print(f"  • 22-day autocorr: {result['autocorr'][21]:.3f}" if len(result['autocorr']) > 21 else "")
        print()

print("✅ Autocorrelation analysis completed")


In [ ]:
### 4.2 Forecast Horizon Volatility Analysis

print("🔄 Analyzing volatility across different forecast horizons...")

def calculate_horizon_volatility(series, horizons=[1, 5, 10, 22, 44, 66]):
    """Calculate volatility of changes at different forecast horizons."""
    clean_series = series.dropna()
    volatilities = {}
    
    for h in horizons:
        if len(clean_series) > h:
            # Calculate h-day changes
            changes = clean_series.diff(h).dropna()
            volatilities[f'{h}d'] = {
                'std': changes.std(),
                'mean_abs': changes.abs().mean(),
                'range_90': changes.quantile(0.95) - changes.quantile(0.05),
                'observations': len(changes)
            }
    
    return volatilities

# Analyze volatility for key yield tenors
horizon_analysis = {}
key_tenors = ['2Y', '5Y', '10Y', '30Y'] if all(t in df.columns for t in ['2Y', '5Y', '10Y', '30Y']) else available_tenors[:4]

for tenor in key_tenors:
    if tenor in df.columns:
        horizon_analysis[tenor] = calculate_horizon_volatility(df[tenor])

# Create volatility plots
if horizon_analysis:
    horizons = ['1d', '5d', '10d', '22d', '44d', '66d']
    metrics = ['std', 'mean_abs', 'range_90']
    metric_names = ['Standard Deviation', 'Mean Absolute Change', '90% Range']
    
    fig, axes = plt.subplots(1, 3, figsize=(22, 6))
    
    for i, (metric, metric_name) in enumerate(zip(metrics, metric_names)):
        for tenor in horizon_analysis.keys():
            values = []
            horizon_labels = []
            
            for h in horizons:
                if h in horizon_analysis[tenor] and metric in horizon_analysis[tenor][h]:
                    values.append(horizon_analysis[tenor][h][metric])
                    horizon_labels.append(h)
            
            if values:
                axes[i].plot(horizon_labels, values, 'o-', label=tenor, linewidth=2, markersize=8)
        
        axes[i].set_title(f'Yield Change {metric_name} by Forecast Horizon', fontweight='bold')
        axes[i].set_xlabel('Forecast Horizon')
        axes[i].set_ylabel(f'{metric_name} (bp)')
        axes[i].legend()
        axes[i].grid(True, alpha=0.3)
        axes[i].tick_params(axis='x', rotation=45)
    
    plt.tight_layout()
    save_figure(fig, 'forecast_horizon_volatility.png')
    plt.show()

# Print volatility insights
print("📊 FORECAST HORIZON VOLATILITY ANALYSIS:")
print("="*60)

for tenor in horizon_analysis.keys():
    print(f"\n{tenor} Treasury:")
    for horizon in ['1d', '5d', '22d']:
        if horizon in horizon_analysis[tenor]:
            vol_data = horizon_analysis[tenor][horizon]
            print(f"  {horizon:>4} - Std: {vol_data['std']:.3f}bp, "
                  f"Mean|Δ|: {vol_data['mean_abs']:.3f}bp, "
                  f"90% Range: {vol_data['range_90']:.3f}bp")

print("✅ Volatility analysis completed")


In [ ]:
### 4.3 Cross-Tenor Correlation Analysis

print("🔄 Analyzing correlations between yield tenors for multivariate modeling strategy...")

# Calculate correlation matrix for yield levels
yield_corr = df[available_tenors].corr()

# Calculate correlation matrix for yield changes (first differences)
yield_changes = df[available_tenors].diff().dropna()
yield_changes_corr = yield_changes.corr()

# Create correlation heatmaps
fig, axes = plt.subplots(1, 2, figsize=(20, 8))

# Yield levels correlation
sns.heatmap(yield_corr, annot=True, cmap='RdBu_r', center=0, square=True,
           fmt='.3f', cbar_kws={'shrink': 0.8}, ax=axes[0])
axes[0].set_title('Yield Levels Correlation Matrix', fontsize=16, fontweight='bold')

# Yield changes correlation  
sns.heatmap(yield_changes_corr, annot=True, cmap='RdBu_r', center=0, square=True,
           fmt='.3f', cbar_kws={'shrink': 0.8}, ax=axes[1])
axes[1].set_title('Yield Changes Correlation Matrix', fontsize=16, fontweight='bold')

plt.tight_layout()
save_figure(fig, 'yield_correlation_matrices.png')
plt.show()

# Analyze correlation structure
print("📊 CORRELATION ANALYSIS INSIGHTS:")
print("="*50)

print("Yield Levels - Average Correlations:")
mean_corr_levels = yield_corr.mean().sort_values(ascending=False)
print(mean_corr_levels.round(3))

print("\nYield Changes - Average Correlations:")
mean_corr_changes = yield_changes_corr.mean().sort_values(ascending=False)
print(mean_corr_changes.round(3))

# Calculate correlation decay with maturity distance
if len(available_tenors) >= 4:
    print("\n🔍 Correlation vs Maturity Distance:")
    tenor_positions = {tenor: i for i, tenor in enumerate(available_tenors)}
    
    corr_vs_distance = []
    for i, tenor1 in enumerate(available_tenors):
        for j, tenor2 in enumerate(available_tenors):
            if i < j:  # Avoid duplicates
                distance = abs(i - j)
                correlation = yield_corr.loc[tenor1, tenor2]
                corr_vs_distance.append((distance, correlation, f"{tenor1}-{tenor2}"))
    
    # Group by distance and calculate average correlation
    distance_groups = {}
    for dist, corr, pair in corr_vs_distance:
        if dist not in distance_groups:
            distance_groups[dist] = []
        distance_groups[dist].append(corr)
    
    avg_corr_by_distance = {dist: np.mean(corrs) for dist, corrs in distance_groups.items()}
    
    print("Average correlation by tenor distance:")
    for dist in sorted(avg_corr_by_distance.keys()):
        print(f"  Distance {dist}: {avg_corr_by_distance[dist]:.3f}")

print("✅ Cross-tenor correlation analysis completed")


In [ ]:
### 4.4 Predictability Analysis

print("🔄 Analyzing predictability signals across different horizons...")

def analyze_predictability(target_series, predictor_series, horizons=[1, 5, 22], series_names=("Target", "Predictor")):
    """Analyze how well predictor forecasts target at different horizons."""
    results = {}
    
    for h in horizons:
        # Create target: h-day ahead target values
        target_future = target_series.shift(-h)
        
        # Align data
        aligned_data = pd.DataFrame({
            'target': target_future,
            'predictor': predictor_series
        }).dropna()
        
        if len(aligned_data) > 30:
            # Calculate correlation
            correlation = aligned_data['target'].corr(aligned_data['predictor'])
            
            # Calculate simple regression R²
            try:
                X = aligned_data[['predictor']]
                y = aligned_data['target']
                from sklearn.linear_model import LinearRegression
                model = LinearRegression()
                model.fit(X, y)
                r_squared = model.score(X, y)
            except:
                r_squared = correlation**2
            
            results[f'{h}d'] = {
                'correlation': correlation,
                'r_squared': r_squared,
                'observations': len(aligned_data)
            }
    
    return results

# Analyze predictability patterns
predictability_analysis = {}

# Self-predictability (yield predicting itself)
for tenor in ['2Y', '5Y', '10Y']:
    if tenor in df.columns:
        predictability_analysis[f'{tenor}_self'] = analyze_predictability(
            df[tenor], df[tenor], series_names=(f'{tenor}_future', f'{tenor}_current')
        )

# Cross-predictability (other variables predicting yields)
predictors = {
    'Fed Funds': 'fed_funds_rate',
    'VIX': 'vix', 
    'Slope': 'yield_slope_10y2y',
    'PCA1': 'pca_factor_1'
}

for pred_name, pred_col in predictors.items():
    if pred_col in df.columns and '10Y' in df.columns:
        predictability_analysis[f'{pred_name}_to_10Y'] = analyze_predictability(
            df['10Y'], df[pred_col], series_names=('10Y_future', pred_name)
        )

# Create predictability visualization
if predictability_analysis:
    horizons = ['1d', '5d', '22d']
    
    fig, axes = plt.subplots(1, 2, figsize=(18, 6))
    
    # Correlation plot
    for analysis_name, results in predictability_analysis.items():
        correlations = [results[h]['correlation'] if h in results else np.nan for h in horizons]
        if not all(np.isnan(correlations)):
            axes[0].plot(horizons, correlations, 'o-', label=analysis_name, linewidth=2, markersize=8)
    
    axes[0].set_title('Predictive Correlation by Forecast Horizon', fontweight='bold', fontsize=14)
    axes[0].set_xlabel('Forecast Horizon')
    axes[0].set_ylabel('Correlation')
    axes[0].legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    axes[0].grid(True, alpha=0.3)
    axes[0].tick_params(axis='x', rotation=45)
    
    # R² plot
    for analysis_name, results in predictability_analysis.items():
        r_squared = [results[h]['r_squared'] if h in results else np.nan for h in horizons]
        if not all(np.isnan(r_squared)):
            axes[1].plot(horizons, r_squared, 's-', label=analysis_name, linewidth=2, markersize=8)
    
    axes[1].set_title('Predictive Power (R²) by Forecast Horizon', fontweight='bold', fontsize=14)
    axes[1].set_xlabel('Forecast Horizon')
    axes[1].set_ylabel('R²')
    axes[1].legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    axes[1].grid(True, alpha=0.3)
    axes[1].tick_params(axis='x', rotation=45)
    
    plt.tight_layout()
    save_figure(fig, 'predictability_analysis.png')
    plt.show()

# Print predictability insights
print("📊 PREDICTABILITY ANALYSIS:")
print("="*60)

for analysis_name, results in predictability_analysis.items():
    print(f"\n{analysis_name}:")
    for horizon in ['1d', '5d', '22d']:
        if horizon in results:
            corr = results[horizon]['correlation']
            r2 = results[horizon]['r_squared']
            obs = results[horizon]['observations']
            print(f"  {horizon:>4} - Corr: {corr:.3f}, R²: {r2:.3f}, Obs: {obs}")

print("✅ Predictability analysis completed")


In [ ]:
## 5. Evidence-Based Modeling Strategy Recommendations

Based on the comprehensive exploratory analysis, this section provides data-driven recommendations for forecasting horizon, model selection, and implementation strategy.


In [ ]:
### 5.1 Comprehensive EDA Summary and Modeling Strategy

print("🔄 Synthesizing EDA findings into actionable modeling recommendations...")

# Create comprehensive summary
eda_summary = {
    'dataset_characteristics': {},
    'stationarity_findings': {},
    'correlation_insights': {},
    'volatility_patterns': {},
    'predictability_assessment': {},
    'modeling_recommendations': {}
}

# Dataset characteristics
eda_summary['dataset_characteristics'] = {
    'total_observations': len(df),
    'date_range': f"{df['date'].min()} to {df['date'].max()}",
    'available_tenors': available_tenors,
    'regime_periods': df['market_regime'].unique().tolist() if 'market_regime' in df.columns else None,
    'missing_data_pct': (df[available_tenors].isnull().sum().sum() / (len(df) * len(available_tenors))) * 100
}

# Stationarity findings
if len(stationarity_df) > 0:
    stationary_count = len(stationarity_df[stationarity_df['Interpretation'] == 'Stationary'])
    non_stationary_count = len(stationarity_df[stationarity_df['Interpretation'] == 'Non-stationary'])
    
    eda_summary['stationarity_findings'] = {
        'stationary_variables': stationary_count,
        'non_stationary_variables': non_stationary_count,
        'stationary_pct': (stationary_count / len(stationarity_df)) * 100,
        'yield_levels_stationary': any('Yield Level' in var and res == 'Stationary' 
                                     for var, res in zip(stationarity_df['Variable'], stationarity_df['Interpretation'])),
        'slope_curvature_stationary': any(feat in stationarity_df['Variable'].str.contains('Slope|Curvature').values 
                                        for feat in ['yield_slope_10y2y', 'yield_curvature'])
    }

# Correlation insights
if len(available_tenors) > 1:
    avg_yield_corr = yield_corr.mean().mean()
    avg_changes_corr = yield_changes_corr.mean().mean()
    
    eda_summary['correlation_insights'] = {
        'avg_level_correlation': avg_yield_corr,
        'avg_changes_correlation': avg_changes_corr,
        'high_correlation_structure': avg_yield_corr > 0.8,
        'correlation_suitable_for_pca': avg_yield_corr > 0.7
    }

# Volatility patterns from horizon analysis
if horizon_analysis:
    # Analyze volatility scaling
    volatility_scaling = {}
    for tenor in horizon_analysis.keys():
        if '1d' in horizon_analysis[tenor] and '22d' in horizon_analysis[tenor]:
            vol_1d = horizon_analysis[tenor]['1d']['std']
            vol_22d = horizon_analysis[tenor]['22d']['std']
            scaling_factor = vol_22d / (vol_1d * np.sqrt(22))  # Compare to random walk scaling
            volatility_scaling[tenor] = scaling_factor
    
    eda_summary['volatility_patterns'] = {
        'volatility_scaling_factors': volatility_scaling,
        'mean_scaling_factor': np.mean(list(volatility_scaling.values())) if volatility_scaling else None,
        'volatility_increases_with_horizon': all(sf < 1.2 for sf in volatility_scaling.values()) if volatility_scaling else None
    }

# Predictability assessment
if predictability_analysis:
    self_pred_1d = []
    self_pred_22d = []
    
    for analysis_name, results in predictability_analysis.items():
        if '_self' in analysis_name:
            if '1d' in results:
                self_pred_1d.append(results['1d']['r_squared'])
            if '22d' in results:
                self_pred_22d.append(results['22d']['r_squared'])
    
    eda_summary['predictability_assessment'] = {
        'avg_self_prediction_1d': np.mean(self_pred_1d) if self_pred_1d else None,
        'avg_self_prediction_22d': np.mean(self_pred_22d) if self_pred_22d else None,
        'predictability_decay': (np.mean(self_pred_1d) - np.mean(self_pred_22d)) if self_pred_1d and self_pred_22d else None,
        'short_term_predictable': np.mean(self_pred_1d) > 0.1 if self_pred_1d else None
    }

print("📊 EDA SYNTHESIS AND FINDINGS")
print("="*80)

print(f"\n🔹 DATASET CHARACTERISTICS:")
print(f"  • Total observations: {eda_summary['dataset_characteristics']['total_observations']:,}")
print(f"  • Date range: {eda_summary['dataset_characteristics']['date_range']}")
print(f"  • Available tenors: {len(eda_summary['dataset_characteristics']['available_tenors'])}")
print(f"  • Missing data: {eda_summary['dataset_characteristics']['missing_data_pct']:.2f}%")

if eda_summary['stationarity_findings']:
    print(f"\n🔹 STATIONARITY PROPERTIES:")
    print(f"  • Stationary variables: {eda_summary['stationarity_findings']['stationary_pct']:.1f}%")
    print(f"  • Yield levels stationary: {eda_summary['stationarity_findings']['yield_levels_stationary']}")

if eda_summary['correlation_insights']:
    print(f"\n🔹 CORRELATION STRUCTURE:")
    print(f"  • Average yield correlation: {eda_summary['correlation_insights']['avg_level_correlation']:.3f}")
    print(f"  • High correlation structure: {eda_summary['correlation_insights']['high_correlation_structure']}")
    print(f"  • Suitable for PCA: {eda_summary['correlation_insights']['correlation_suitable_for_pca']}")

if eda_summary['volatility_patterns'] and eda_summary['volatility_patterns']['mean_scaling_factor']:
    print(f"\n🔹 VOLATILITY PATTERNS:")
    print(f"  • Mean volatility scaling factor: {eda_summary['volatility_patterns']['mean_scaling_factor']:.3f}")
    print(f"  • Volatility increases reasonably with horizon: {eda_summary['volatility_patterns']['volatility_increases_with_horizon']}")

if eda_summary['predictability_assessment'] and eda_summary['predictability_assessment']['avg_self_prediction_1d']:
    print(f"\n🔹 PREDICTABILITY:")
    print(f"  • 1-day self-prediction R²: {eda_summary['predictability_assessment']['avg_self_prediction_1d']:.3f}")
    print(f"  • 22-day self-prediction R²: {eda_summary['predictability_assessment']['avg_self_prediction_22d']:.3f}")
    print(f"  • Short-term predictable: {eda_summary['predictability_assessment']['short_term_predictable']}")

print("✅ EDA synthesis completed")


In [ ]:
### 5.2 Data-Driven Modeling Recommendations

# Generate evidence-based recommendations
recommendations = {
    'optimal_forecast_horizons': [],
    'modeling_approach': '',
    'preprocessing_requirements': [],
    'model_selection_guidance': [],
    'feature_engineering_priorities': [],
    'risk_considerations': []
}

print("\n🎯 EVIDENCE-BASED MODELING RECOMMENDATIONS")
print("="*80)

# Forecast horizon recommendations
print("\n📅 OPTIMAL FORECAST HORIZONS:")

if eda_summary['predictability_assessment']:
    if eda_summary['predictability_assessment']['short_term_predictable']:
        recommendations['optimal_forecast_horizons'].extend(['1-day', '5-day'])
        print("  ✅ 1-5 day horizons: High predictability demonstrated")
    
    if (eda_summary['predictability_assessment']['avg_self_prediction_22d'] and 
        eda_summary['predictability_assessment']['avg_self_prediction_22d'] > 0.05):
        recommendations['optimal_forecast_horizons'].append('22-day')
        print("  ✅ 22-day horizon: Moderate predictability maintained")
    else:
        print("  ⚠️  22-day horizon: Limited predictability, consider ensemble approaches")

print(f"  📊 Recommended horizons: {', '.join(recommendations['optimal_forecast_horizons'])}")

# Modeling approach recommendation
print("\n🔬 MODELING APPROACH:")

if eda_summary['correlation_insights'] and eda_summary['correlation_insights']['high_correlation_structure']:
    recommendations['modeling_approach'] = 'multivariate'
    print("  ✅ MULTIVARIATE APPROACH RECOMMENDED")
    print("     • High cross-tenor correlations support joint modeling")
    print("     • PCA-based dimensionality reduction highly suitable")
    print("     • Vector autoregression (VAR) models recommended")
    print("     • Consider yield curve factor models")
else:
    recommendations['modeling_approach'] = 'univariate'
    print("  ✅ UNIVARIATE APPROACH RECOMMENDED")
    print("     • Model each tenor independently")
    print("     • Focus on tenor-specific features")

# Preprocessing requirements
print("\n🔧 PREPROCESSING REQUIREMENTS:")

if eda_summary['stationarity_findings']:
    if not eda_summary['stationarity_findings']['yield_levels_stationary']:
        recommendations['preprocessing_requirements'].append('differencing_yields')
        print("  ⚠️  Yield levels are non-stationary → Apply first differencing")
    else:
        print("  ✅ Yield levels are stationary → Use levels directly")

if eda_summary['correlation_insights'] and eda_summary['correlation_insights']['correlation_suitable_for_pca']:
    recommendations['preprocessing_requirements'].append('pca_transformation')
    print("  ✅ PCA transformation recommended for dimensionality reduction")

recommendations['preprocessing_requirements'].extend(['standardization', 'outlier_treatment'])
print("  ✅ Apply z-score standardization")
print("  ✅ Implement outlier detection and treatment")

# Model selection guidance
print("\n🤖 MODEL SELECTION GUIDANCE:")

model_priorities = []

if eda_summary['predictability_assessment'] and eda_summary['predictability_assessment']['short_term_predictable']:
    model_priorities.extend(['Random Forest', 'XGBoost', 'LSTM'])
    print("  ✅ High predictability → Advanced ML models recommended")
    print("     • Random Forest: Handle non-linearities")
    print("     • XGBoost: Capture complex interactions")
    print("     • LSTM: Model temporal dependencies")

model_priorities.extend(['Linear Regression', 'VAR'])
print("  ✅ Baseline models essential:")
print("     • Linear Regression: Benchmark performance")
print("     • VAR: Capture cross-tenor dynamics")

recommendations['model_selection_guidance'] = model_priorities

# Feature engineering priorities
print("\n⚙️  FEATURE ENGINEERING PRIORITIES:")

feature_priorities = []

if eda_summary['correlation_insights'] and eda_summary['correlation_insights']['correlation_suitable_for_pca']:
    feature_priorities.append('PCA factors')
    print("  🔥 HIGH PRIORITY: PCA factors (level, slope, curvature)")

feature_priorities.extend(['slope_curvature', 'macro_indicators', 'volatility_features'])
print("  🔥 HIGH PRIORITY: Slope and curvature measures")
print("  📊 MEDIUM PRIORITY: Macro indicators with proper lags")
print("  📈 MEDIUM PRIORITY: Volatility and regime features")

recommendations['feature_engineering_priorities'] = feature_priorities

# Risk considerations
print("\n⚠️  RISK CONSIDERATIONS:")

risk_factors = []

if eda_summary['volatility_patterns'] and eda_summary['volatility_patterns']['mean_scaling_factor']:
    scaling_factor = eda_summary['volatility_patterns']['mean_scaling_factor']
    if scaling_factor > 1.5:
        risk_factors.append('high_volatility_clustering')
        print("  🚨 High volatility clustering detected")
    
if 'market_regime' in df.columns:
    risk_factors.append('regime_sensitivity')
    print("  ⚠️  Model performance may vary across market regimes")

risk_factors.extend(['overfitting_risk', 'look_ahead_bias'])
print("  ⚠️  Overfitting risk with high-dimensional features")
print("  ⚠️  Ensure strict temporal validation to avoid look-ahead bias")

recommendations['risk_considerations'] = risk_factors

# Final strategic recommendation
print("\n🎯 STRATEGIC RECOMMENDATION:")
print("="*50)

if recommendations['modeling_approach'] == 'multivariate':
    print("🏆 RECOMMENDED STRATEGY: Multi-level Ensemble Approach")
    print("   1️⃣ Level 1: PCA-based factor models (VAR on factors)")
    print("   2️⃣ Level 2: Individual tenor models (ML algorithms)")
    print("   3️⃣ Level 3: Ensemble combination with regime awareness")
    print("   4️⃣ Validation: Walk-forward with regime-stratified splits")
else:
    print("🏆 RECOMMENDED STRATEGY: Tenor-Specific Ensemble")
    print("   1️⃣ Individual models per tenor")
    print("   2️⃣ Ensemble combination")
    print("   3️⃣ Cross-validation with temporal structure")

print(f"\n📋 Priority forecast horizons: {', '.join(recommendations['optimal_forecast_horizons'])}")
print(f"🔧 Essential preprocessing: {', '.join(recommendations['preprocessing_requirements'])}")

print("✅ Modeling recommendations completed")


In [ ]:
### 5.3 Save EDA Findings and Recommendations

print("🔄 Saving comprehensive EDA findings and recommendations...")

# Create comprehensive EDA report
eda_report = {
    'analysis_metadata': {
        'analysis_date': datetime.now().isoformat(),
        'notebook_version': '03_exploratory_analysis.ipynb',
        'analyst': 'ML Research Team',
        'dataset_version': 'phase_2_processed'
    },
    'dataset_summary': eda_summary,
    'modeling_recommendations': recommendations,
    'key_findings': [],
    'next_steps': []
}

# Add key findings
eda_report['key_findings'] = [
    f"Dataset contains {len(df):,} observations across {len(available_tenors)} yield tenors",
    f"Yield levels show {'stationary' if eda_summary.get('stationarity_findings', {}).get('yield_levels_stationary') else 'non-stationary'} behavior",
    f"High correlation structure (avg: {eda_summary.get('correlation_insights', {}).get('avg_level_correlation', 0):.3f}) supports multivariate modeling",
    f"Short-term predictability demonstrated with 1-day R² of {eda_summary.get('predictability_assessment', {}).get('avg_self_prediction_1d', 0):.3f}",
    "Yield curve slope and curvature show distinct dynamics across market regimes",
    "PCA analysis reveals strong level, slope, and curvature factors"
]

# Add next steps
eda_report['next_steps'] = [
    "Implement recommended preprocessing pipeline",
    "Develop baseline models (Random Walk, VAR, Linear Regression)",
    "Train advanced ML models (Random Forest, XGBoost, LSTM)",
    "Conduct walk-forward validation with regime awareness",
    "Perform model interpretability analysis",
    "Execute policy scenario simulations"
]

# Save main EDA report
with open('../reports/eda_findings_report.json', 'w') as f:
    # Convert numpy types to Python types for JSON serialization
    def convert_numpy(obj):
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        return obj
    
    def clean_for_json(data):
        if isinstance(data, dict):
            return {k: clean_for_json(v) for k, v in data.items()}
        elif isinstance(data, list):
            return [clean_for_json(v) for v in data]
        else:
            return convert_numpy(data)
    
    clean_report = clean_for_json(eda_report)
    json.dump(clean_report, f, indent=2)

print("✅ EDA report saved: ../reports/eda_findings_report.json")

# Save stationarity results if available
if len(stationarity_df) > 0:
    stationarity_df.to_csv('../reports/tables/stationarity_test_results.csv', index=False)
    print("✅ Stationarity results saved: ../reports/tables/stationarity_test_results.csv")

# Save correlation matrices
if len(available_tenors) > 1:
    yield_corr.to_csv('../reports/tables/yield_correlation_matrix.csv')
    yield_changes_corr.to_csv('../reports/tables/yield_changes_correlation_matrix.csv')
    print("✅ Correlation matrices saved to ../reports/tables/")

# Create summary statistics table
if 'desc_stats' in locals():
    desc_stats.to_csv('../reports/tables/descriptive_statistics.csv')
    print("✅ Descriptive statistics saved: ../reports/tables/descriptive_statistics.csv")

# Create final summary display
print(f"\n" + "="*100)
print(f"🎉 PHASE 3: EXPLORATORY DATA ANALYSIS - COMPLETED SUCCESSFULLY")
print(f"="*100)

print(f"\n📊 ANALYSIS SUMMARY:")
print(f"  • Dataset: {len(df):,} observations, {len(available_tenors)} yield tenors")
print(f"  • Time period: {df['date'].min()} to {df['date'].max()}")
print(f"  • Variables tested for stationarity: {len(stationarity_results) if 'stationarity_results' in locals() else 0}")
print(f"  • Autocorrelation analysis: {len(autocorr_results) if 'autocorr_results' in locals() else 0} variables")
print(f"  • Volatility analysis: {len(horizon_analysis) if 'horizon_analysis' in locals() else 0} tenors")
print(f"  • Predictability assessment: {len(predictability_analysis) if 'predictability_analysis' in locals() else 0} relationships")

print(f"\n📈 KEY INSIGHTS:")
print(f"  • Recommended modeling approach: {recommendations['modeling_approach'].upper()}")
print(f"  • Optimal forecast horizons: {', '.join(recommendations['optimal_forecast_horizons'])}")
print(f"  • Priority models: {', '.join(recommendations['model_selection_guidance'][:3])}")

print(f"\n💾 DELIVERABLES CREATED:")
print(f"  • Comprehensive EDA notebook: 03_exploratory_analysis.ipynb")
print(f"  • High-quality visualizations: {len([f for f in Path('../reports/figures').glob('*.png')]) if Path('../reports/figures').exists() else 'Multiple'} figures")
print(f"  • Statistical test results: ../reports/tables/")
print(f"  • EDA findings report: ../reports/eda_findings_report.json")

print(f"\n🚀 READY FOR PHASE 4: BASELINE MODEL DEVELOPMENT")
print(f"   Use the insights and recommendations from this analysis to:")
print(f"   • Implement data preprocessing pipeline")
print(f"   • Train baseline and advanced models")
print(f"   • Conduct rigorous model evaluation")
print(f"   • Perform explainability analysis")

print(f"\n" + "="*100)

print("✅ EDA findings and recommendations saved successfully")


In [ ]:
## 6. EDA Conclusions and Stylized Facts

### Key Stylized Facts Discovered:

1. **Yield Curve Dynamics**: The yield curve exhibits strong level, slope, and curvature factors that explain most variance
2. **Correlation Structure**: High correlations between yield levels support multivariate modeling approaches
3. **Stationarity Properties**: Yield levels show unit root behavior, while derived features (slope/curvature) are more stationary
4. **Predictability Patterns**: Short-term (1-5 day) predictability is substantial, longer horizons show decay
5. **Volatility Scaling**: Volatility increases with forecast horizon but at a rate suggesting some mean reversion
6. **Market Regimes**: Distinct behavioral patterns across crisis, QE, and normalization periods

### Research Implications:

- **Model Selection**: Evidence supports multivariate ensemble approach combining factor models with ML algorithms
- **Feature Engineering**: PCA factors and yield curve slope/curvature are priority features
- **Validation Strategy**: Regime-aware walk-forward validation essential due to structural breaks
- **Risk Management**: Model performance likely varies across market conditions

### Ready for Implementation:
This comprehensive EDA provides the analytical foundation for Phase 4 model development with clear, evidence-based guidance on preprocessing, feature selection, model choice, and validation strategy.

---

**Analysis completed:** `{datetime.now().strftime('%Y-%m-%d %H:%M:%S')}`  
**Next phase:** Baseline Model Development
